In [38]:
import pandas as pd
import numpy as np
csv_link = "https://covid19-lake.s3.us-east-2.amazonaws.com/enigma-aggregation/csv/global_countries/enigma_covid_19_global_countries.csv"
df_raw = pd.read_csv(csv_link)
df_raw

,country_name,country_iso2,country_iso3,lat,long,population,date,cases,deaths,tests,tests_units
0,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-25,769,22,38146,tests performed
1,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-26,787,22,38695,tests performed
2,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-27,789,22,39469,tests performed
3,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-28,803,22,40229,tests performed
4,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-29,811,22,41469,tests performed
...,...,...,...,...,...,...,...,...,...,...,...
24942,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-20,738,20,34794,tests performed
24943,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-21,746,20,0,tests performed
24944,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-22,749,20,35537,tests performed
24945,Uruguay,UY,URY,-32.522779,-55.765835,3473727,2020-05-23,753,20,37178,tests performed


In [65]:
# increase_rate(date) = (num_today - num_yesterday) / num_yesterday
def increase_rate(df_raw, country, metric):
    df = df_raw[df_raw['country_name'] == country][['country_name', 'date', metric]].reset_index(drop=True)
    df['increase_rate'] = df.groupby('country_name')[[metric]].pct_change()
    df.fillna(0, inplace=True)
    df.replace([np.inf], 0, inplace=True)
    res = df
    return res

df_increase_rate = increase_rate(df_raw, 'Germany', 'cases')
df_increase_rate

,country_name,date,cases,increase_rate
0,Germany,2019-12-31,0,0.000000
1,Germany,2020-01-01,0,0.000000
2,Germany,2020-01-02,0,0.000000
3,Germany,2020-01-03,0,0.000000
4,Germany,2020-01-04,0,0.000000
...,...,...,...,...
169,Germany,2020-06-17,187184,0.001847
170,Germany,2020-06-18,187764,0.003099
171,Germany,2020-06-19,188534,0.004101
172,Germany,2020-06-20,189135,0.003188


In [64]:

# avg_increase_rate_before_event(event_time) = avg(increase_rate(origin to event))
# avg_increase_rate_after_event(event_time) = avg(increase_rate(event to today))

def avg_increase_rate(df_with_increase_rate, event_date, after=True):
    df = df_with_increase_rate
    df_event = df[df['date'] > event_date]
    if not after:
        df_event = df[df['date'] <= event_date]
    res = df_event['increase_rate'].mean()
    return res

avg_after_childrensday = avg_increase_rate(df_increase_rate, '2020-05-01')
avg_before_childrensday = avg_increase_rate(df_increase_rate, '2020-05-01', False)
print(avg_after_childrensday)
print(avg_before_childrensday)

0.0034684549291337328
0.12630508536468352


In [3]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd

dashboard_name = "COVID-19 Dashboard"
info = 'Germany'
csv_link = "https://covid19-lake.s3.us-east-2.amazonaws.com/enigma-aggregation/csv/global_countries/enigma_covid_19_global_countries.csv"
df_raw = pd.read_csv(csv_link)
df_usa = df_raw[df_raw['country_name'] == 'United States'][['date', 'cases', 'deaths', 'tests']].reset_index(drop=True)
timeline_info = "Timeline from https://www.nytimes.com/article/coronavirus-timeline.html"


def init(app):
    app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

    metric_type_dropdown = dcc.Dropdown(
        id='metric_type',
        options=[
            {'label': 'Cases', 'value': 'cases'},
            {'label': 'Deaths', 'value': 'deaths'},
        ],
        value='cases'
    )

    country_dropdown = dcc.Dropdown(
        id='country',
        options=[{'label': country, 'value': country} for country in df_raw['country_name'].unique()],
        value='Germany'
    )

    measure_dropdown = dcc.Dropdown(
        id='measure',
        options=[{'label': 'Obligation to notify', 'value': '2020-01-31'},
                 {'label': 'Stop of entry', 'value': '2020-03-17'},
                 {'label': 'Contact restrictions', 'value': '2020-03-22'},
                 {'label': 'Quarantine obligation', 'value': '2020-04-10'},
                 {'label': 'Masc obligation', 'value': '2020-04-22'},
                 {'label': 'Strong relaxations of Corona measures', 'value': '2020-05-06'}, ],
        value='Obligation to notify',
    )

    app.layout = html.Div([
        html.H1(children=dashboard_name),
        html.H4(children=f'datasource: {csv_link}'),
        html.H6(children=info),
        html.H2(children="Select metric type"),
        metric_type_dropdown,
        html.H2(children="Select country"),
        country_dropdown,
        html.H2(children="Select measure"),
        measure_dropdown,

        dcc.Graph(
            id='my-graph',
            figure={
                'data': [
                    {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                    {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
                ],
                'layout': {
                    'title': 'Dash Data Visualization'
                }
            }
        )])


if __name__ == '__main__':
    app = dash.Dash(dashboard_name)
    init(app)
    app.run_server(host='0.0.0.0', port=8050, debug=True)


Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

Dash is running on http://0.0.0.0:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "COVID-19 Dashboard" (lazy loading)
 * Environment: development
 * Debug mode: on


SystemExit: 1

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
